In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy.ma as ma
from numpy import meshgrid
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm

import geopandas as gpd
from shapely.geometry  import Point
from shapely.geometry import shape
import seaborn as sns


In [ ]:
path_out_P = '/home/vikram/BCSD_data/' + 'US_P_BCSD/'
path_out_T = '/home/vikram/BCSD_data/' + 'US_T_BCSD/'

output_figname = ''#_eqm'

# init

In [ ]:
gcm_names=['MIROC_ESM','CanESM2','HadGEM_AO','GFDL_CM3']
gcm_name  =  gcm_names[1]

In [ ]:
def getclosest_ij(lats,lons,latpt,lonpt):
    dis_sq1=(lats-latpt)**2
    dis_sq2=(lons-lonpt)**2
    
    minidex_lat=dis_sq1.argmin()
    minidex_lon=dis_sq2.argmin()
    
    return minidex_lon,minidex_lat

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import geopandas as gpd
from geopy.geocoders import Nominatim



lats,lons = (np.arange(0,40,1),np.arange(-180,100,1))
XX12,YY12 = np.meshgrid(lons,lats)

fig, ax = plt.subplots()

for address in ['San Francisco','New York','Los Angeles', 'Chicago']:
    geolocator = Nominatim(user_agent="Your_Name")
    location = geolocator.geocode(address)
    [ind_lon,ind_lat] = getclosest_ij(lats,lons,location.latitude,location.longitude)
    plt.plot(lons[ind_lon],lats[ind_lat],marker='*',markersize= 25)

ind = gpd.read_file('/home/vikram/BCSD_Shapefiles/Us_boundary.shp')
ind.plot(ax=ax,facecolor="none")

regional_boxes = [[-120,37,-113,45],
                  [-109,37,-98,45],
                  [-88,33,-77,41]]

for co_ords in regional_boxes:
    ax.add_patch(Polygon([(co_ords[0],co_ords[1]),
                           (co_ords[0],co_ords[3]),
                           (co_ords[2],co_ords[3]),
                           (co_ords[2],co_ords[1])],facecolor='none',edgecolor= 'k'))

plt.show()




# Import data

In [ ]:

metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
st=metadata.item()['st']
en=metadata.item()['en']

mask_fine_obs=metadata.item()['mask']

lat_obs=metadata.item()['lat_obs']
lon_obs=metadata.item()['lon_obs']

N_random = 10
gcm_list = ['GFDL_CM3','CanESM2','MIROC_ESM','MRI_ESM1','ACCESS1_0','NorESM1_M','CM5A_LR','CNRM_CM5','MPI_ESM_MR']
N_esms = len(gcm_list)
tas_obs=np.load(path_out_T+ '/tas_obs.npy')
tas_data_all=np.empty([1+N_esms *4,tas_obs.shape[0],tas_obs.shape[1],tas_obs.shape[2]])
pre_obs=np.load(path_out_P+ '/pre_obs.npy')
pre_data_all=np.empty([1+N_esms * 4,pre_obs.shape[0],pre_obs.shape[1],pre_obs.shape[2]])


tas_data_all[0]=tas_obs
pre_data_all[0]=pre_obs

for giter,gcm_name in tqdm(enumerate(gcm_list)):
    tas_data_all[1+giter]=np.load(path_out_T+ '/tas_gcm_fine_'+gcm_name+'.npy')
    tas_data_all[1+giter +N_esms]=np.load(path_out_T+ '/detrended_BCSD_outputs_historical_tas_'+gcm_name+'_cru.npy')
    
    tas_gcm=np.load(path_out_T+ '/tas_gcm_future_'+gcm_name+'.npy')
    metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
    
    lat_gcm=metadata.item()['lat_gcm']
    lon_gcm=metadata.item()['lon_gcm']
   
    tas_gcm_fine=np.zeros((tas_gcm.shape[0],tas_obs.shape[1],tas_obs.shape[2]))
    
    for i in range(tas_gcm.shape[0]):
        tas_gcm_fine[i,:,:]=uu.regrid(tas_gcm[i,:,:],lat_gcm,lon_gcm,lat_obs,lon_obs)
        

    tas_data_all[1+giter + 2* N_esms]        = np.concatenate((tas_gcm_fine  , np.nan * np.zeros((120 ,tas_obs.shape[1],tas_obs.shape[2]))),axis = 0)
    temp_123 =(np.load(path_out_T+ '/detrended_BCSD_outputs_tas_'+gcm_name+'_cru.npy'))
    tas_data_all[1+giter + 3* N_esms] = np.concatenate((temp_123  ,
                                                   np.nan * np.zeros((120 ,tas_obs.shape[1],tas_obs.shape[2]))),axis = 0)
      
    
    
    
    pre_data_all[1 + giter ]=np.load(path_out_P+ '/pre_gcm_fine'+gcm_name+'.npy')
    pre_data_all[1 + giter +N_esms]=np.load(path_out_P+ '/detrended_BCSD_outputs_historical_pre_'+gcm_name+'_cru.npy')
    
    pre_gcm=np.load(path_out_P+ '/pre_gcm_future_'+gcm_name+'.npy')
    metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
    
    lat_gcm=metadata.item()['lat_gcm']
    lon_gcm=metadata.item()['lon_gcm']
   
    pre_gcm_fine=np.zeros((pre_gcm.shape[0],pre_obs.shape[1],pre_obs.shape[2]))

    for i in range(pre_gcm.shape[0]):
        pre_gcm_fine[i,:,:]=uu.regrid(pre_gcm[i,:,:],lat_gcm,lon_gcm,lat_obs,lon_obs)
        
    pre_data_all[1 + giter + 2 * N_esms ]= np.concatenate((pre_gcm_fine  , np.nan * np.zeros((120 ,pre_obs.shape[1],pre_obs.shape[2]))),axis = 0)
    pre_data_all[1 + giter  + 3 * N_esms]= np.concatenate((uu.correct_nan_inf_neg(np.load(path_out_P+ '/detrended_BCSD_outputs_pre_'+gcm_name+'_cru.npy'))  ,
                                                    np.nan * np.zeros((120 ,pre_obs.shape[1],pre_obs.shape[2]))),axis = 0)
                                    
        
    




In [ ]:
def create_mask(fo,lat_obs_cru,lon_obs_cru,buffer=0,plot=0):
  filename=gpd.read_file(fo)
  shp_geom = shape(filename.geometry[0])

  xxc, yyc = meshgrid(lon_obs_cru,lat_obs_cru)
  mask_shape=np.zeros(xxc.shape,dtype=bool)
  if plot==1:
    x,y = shp_geom.exterior.xy
    plt.plot(x,y)
  N1=xxc.shape[0]
  N2=xxc.shape[1]

  for lat_i in range(0,N1):
    for lon_i in range(0,N2):
      xi=xxc[lat_i,lon_i]
      yi=yyc[lat_i,lon_i]
      m1= (shp_geom.contains(Point(xi, yi)) or shp_geom.touches(Point(xi, yi)))
      mask_shape[lat_i,lon_i]=not m1

  return mask_shape

In [ ]:
ind = gpd.read_file('/home/vikram/BCSD_Shapefiles/Us_boundary.shp')
ind.plot(ax=ax,facecolor="none")
mask_usf=create_mask('/home/vikram/BCSD_Shapefiles/us_rough.shp',lat_obs.data,lon_obs.data-360,buffer=1,plot=0)



In [ ]:
mask_usf_int = np.array(mask_usf,dtype= np.float16)
mask_usf_int[mask_usf == True] = np.nan
mask_usf_int[mask_usf == False] = 1
plt.pcolor(mask_usf_int)
plt.colorbar()

In [ ]:
plt.figure(figsize =(15,20))

ax = plt.subplot(N_esms+1,4,1)
dat_monthly = pre_data_all[0].reshape((12,-1,52,112),order ='F')
dat_annual = np.sum(dat_monthly,axis = 0)

plt.pcolor(lon_obs,lat_obs,np.mean(dat_annual,axis=0)*mask_usf_int,cmap = 'BrBG',
          vmax= 2000,vmin = 0)
plt.colorbar()
plt.text(0.995, 0.995, "Observation", horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
       fontsize=18,color='k')

siter=0
for nrow in range(4):
    for ncol  in range(N_esms):#col is reaally a row xd
        
        itr =  nrow + ncol*4 + 1
        
        ax =plt.subplot(N_esms +1,4,4+itr)
        
        siter=siter+1
        
        
        dat_monthly = pre_data_all[siter].reshape((12,-1,52,112),order ='F')
        dat_annual = np.sum(dat_monthly,axis = 0)
        
        plt.pcolor(lon_obs,lat_obs,np.nanmean(dat_annual,axis=0)*mask_usf_int,cmap = 'BrBG',
          vmax= 2000,vmin = 0)
        plt.colorbar()
        # break
          
        plt.text(0.995, 0.995, gcm_list[ncol], horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
               fontsize=18,color='k')
        
        
plt.suptitle("Mean of Annual rainfall")
plt.tight_layout()## change to monthly
## plot mean and Inter annual variability
plt.savefig(path_out +  "map_mean_us.png", dpi =150, facecolor = 'w')


In [ ]:
plt.figure(figsize =(15,20))

ax = plt.subplot(N_esms+1,4,1)
dat_monthly = pre_data_all[0].reshape((12,-1,52,112),order ='F')
dat_annual = np.sum(dat_monthly,axis = 0)

plt.pcolor(lon_obs,lat_obs,np.nanvar(dat_annual,axis=0)*mask_usf_int/10000,cmap = 'RdYlBu_r',
          vmax= 10,vmin = 0)
plt.colorbar()
plt.text(0.995, 0.995, "Observation", horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
       fontsize=18,color='k')

siter=0
for nrow in range(4):
    for ncol  in range(N_esms):#col is reaally a row xd
        
        itr =  nrow + ncol*4 + 1
        
        ax =plt.subplot(N_esms +1,4,4+itr)
        
        siter=siter+1
        
        
        dat_monthly = pre_data_all[siter].reshape((12,-1,52,112),order ='F')
        dat_annual = np.sum(dat_monthly,axis = 0)
        
        plt.pcolor(lon_obs,lat_obs,np.nanvar(dat_annual,axis=0)*mask_usf_int/10000,cmap = 'RdYlBu_r',
          vmax= 10,vmin = 0)
        plt.colorbar()
        # break
          
        plt.text(0.995, 0.995, gcm_list[ncol], horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
               fontsize=18,color='k')
        
        
plt.suptitle("IAV of Annual rainfall")
plt.tight_layout()## change to monthly
## plot mean and Inter annual variability
plt.savefig(path_out +  "map_var_us.png", dpi =150, facecolor = 'w')


In [ ]:
def extract_regions(data_input,lat_obs,lon_obs):
    
    data_to_indices = []
    for address in ['San Francisco','New York','Los Angeles', 'Chicago']:
        geolocator = Nominatim(user_agent="Your_Name")
        location = geolocator.geocode(address)
        [ind_lon,ind_lat] = getclosest_ij(lat_obs.data,lon_obs.data,location.latitude%360,location.longitude%360)
        data_to_indices.append(data_input[:, ind_lat,ind_lon])


    regional_boxes = [[-120%360,45,-113%360,37],
                      [-109%360,45,-98%360,37],
                      [-88%360,41,-77%360,33]]
    
    
    for co_ords in regional_boxes:

        lat_index_123 = np.logical_and(lat_obs >= co_ords[3], lat_obs <= co_ords[1])
        lon_index_123 = np.logical_and(lon_obs >= co_ords[0], lon_obs <= co_ords[2])
        # print(lat_index_123.sum(), lon_index_123.sum())
        data_to_indices.append(np.nanmean(data_input[:, lat_index_123,:][:,:,lon_index_123],axis=(1,2)))

    data_to_indices.append(np.nanmean(data_input,axis=(1,2)))

    # print(data_to_indices[0].shape)
    data_out= {}
    data_out['Mean'] = []
    data_out['Var'] = []
    
    
    for data_iter in (data_to_indices):
        data_temp =  data_iter.reshape((12,-1),order ='F').sum(axis = 0)
        data_out['Mean'].append(np.mean(data_temp))
        data_out['Var'].append(np.var(data_temp))
        
    return data_out


In [ ]:
data_mean_all = []
data_var_all = []
for diter in tqdm(range(1+N_esms*4)):
    dataset =pre_data_all[diter]
    data_mean_all.append(extract_regions(dataset[45*12:-120,:,:],lat_obs,lon_obs)['Mean'])
    data_var_all.append(extract_regions(dataset[45*12:-120,:,:],lat_obs,lon_obs)['Var'])


In [ ]:

plt.figure(figsize = (9,4))

N_esm =len(gcm_list)
N_random = 10
    
markers=['o','^','s','p','P','*','h','+','x','X','D']

data_to_indices_label = ['San Francisco','New York','Los Angeles', 'Chicago', 'West', 'Middle', 'East' ,"CONUS"]
path_out = "/home/vikram/Dropbox/BCSD Work/03_Figures/plots_Oct10/"

for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot( 1, data_mean_all[0][pliter],
         markers[0],markersize = 10,linestyle = 'None',label = "Obs")


        
        
    for gi in range(N_esm):
        
        
        plt.plot( 2 , data_mean_all[1+gi][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , data_mean_all[1+gi+N_esm][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        plt.plot( 4 , data_mean_all[1+gi+N_esm*2][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none')
        
        plt.plot( 5 , data_mean_all[1+gi+N_esm*3][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        

    
    plt.ylim([0,1800])
    

    plt.xticks(range(1,6),["OBS","ESM-H","BCSD-H","ESM-F","BCSD-F"],rotation = 90 )
    plt.title(data_to_indices_label[pliter])
    
    plt.xlim([0.5,5.5])
    plt.ylabel("mm")
    
plt.suptitle("Mean of Annual rainfall")
plt.tight_layout()
plt.savefig(path_out +  "mean_us_citreg.png", dpi =150, facecolor = 'w')


In [ ]:

plt.figure(figsize = (9,4))

N_esm =len(gcm_list)
    
markers=['o','^','s','p','P','*','h','+','x','X','D']

data_to_indices_label = ['San Francisco','New York','Los Angeles', 'Chicago', 'West', 'Middle', 'East' ,"CONUS"]
path_out = "/home/vikram/Dropbox/BCSD Work/03_Figures/plots_Oct10/"

for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot( 1, data_var_all[0][pliter]/10000,
         markers[0],markersize = 10,linestyle = 'None',label = "Obs")


        
        
    for gi in range(N_esm):
        
        
        plt.plot( 2 , data_var_all[1+gi][pliter]/10000,
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , data_var_all[1+gi+N_esm][pliter]/10000,
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        plt.plot( 4 , data_var_all[1+gi+N_esm*2][pliter]/10000,
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none')
        
        plt.plot( 5 , data_var_all[1+gi+N_esm*3][pliter]/10000,
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
        
        

    
    if pliter < 3:
        plt.ylim([0,10])
    elif pliter < 4:
        plt.ylim([0,5])
    elif pliter < 7:
        plt.ylim([0,3])
    else:
        plt.ylim([0,0.5])
    

    plt.xticks(range(1,6),["OBS","ESM-H","BCSD-H","ESM-F","BCSD-F"],rotation = 90 )
    plt.title(data_to_indices_label[pliter])
    
    plt.xlim([0.5,5.5])
    plt.ylabel("$x10^4 mm^2$")
plt.suptitle("IAV of Annual rainfall")
plt.tight_layout()
plt.savefig(path_out +  "var_us_citreg.png", dpi =150, facecolor = 'w')
